# VisDoMRAG Qwen Notebook

`visdomrag/` 패키지를 사용해 Qwen 기반 VisDoMRAG 파이프라인을 단계별로 실행하는 템플릿입니다.

## 1. 환경 설정
필요한 패키지를 설치하고, 프로젝트 루트를 파이썬 경로에 추가합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

ROOT = Path('/Users/gimchaeyeon/Documents/2025/GL/VisDoM-main').resolve()


if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

ROOT

## 2. 모듈 임포트
`visdomrag` 패키지에서 제공하는 헬퍼들을 불러옵니다.

In [ ]:
import json
from pprint import pprint

import pandas as pd

from visdomrag import (
    VisDoMRAGConfig,
    load_dataset,
    RetrievalManager,
    init_qwen,
    process_query,
    run_pipeline,
)

## 3. 실험 설정값 정의
데이터/출력 경로와 리트리버 종류 등을 필요에 맞게 수정하세요.

In [ ]:
DATA_DIR = ROOT / 'feta_tab'
OUTPUT_DIR = ROOT / 'outputs' / 'feta_tab_qwen'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

config = VisDoMRAGConfig(
    data_dir=DATA_DIR,
    output_dir=OUTPUT_DIR,
    vision_retriever='colpali',
    text_retriever='bm25',
    top_k=5,
    force_reindex=False,
)
config.ensure_directories()
config

## 4. 데이터 로드 & 리트리버 초기화
CSV를 DataFrame으로 읽고 RetrieverManager가 모델을 준비하도록 합니다.

In [ ]:
df = load_dataset(config)
retrieval = RetrievalManager(config=config, df=df)
len(df)

## 5. Qwen 리소스 로드
GPU 메모리를 사용하므로 한 번만 초기화하고 계속 재사용하세요.

In [ ]:
# qwen = init_qwen(model_name='Qwen/Qwen2-VL-7B-Instruct')
qwen = init_qwen(model_name='Qwen/Qwen2-VL-2B-Instruct')
qwen.device

## 6. 단일 쿼리 테스트
원하는 `q_id`를 입력해 전체 파이프라인을 검증합니다.

In [ ]:
sample_id = df['q_id'].iloc[0]
sample_id

In [ ]:
success = process_query(config, retrieval, qwen, query_id=sample_id)
print('Success:', success)
success

## 7. 다수 쿼리 일괄 실행
샘플 q_id 목록을 구성해 `run_pipeline`을 호출합니다.

In [ ]:
subset = df['q_id'].sample(n=5, random_state=42).tolist()
results = run_pipeline(config, retrieval, qwen, query_ids=subset, delay_seconds=0)
results

## 8. 결과 확인
생성된 JSON 파일을 열어 Qwen 응답을 점검합니다.

In [ ]:
result_path = config.combined_output_dir / f"response_{sample_id.replace('/', '$')}.json"
payload = json.loads(result_path.read_text())
pprint(payload)

## 9. 메모리 정리 (선택)
장시간 실험 시 CUDA 메모리를 비워줍니다.

In [ ]:
import gc
import torch

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()